In [2]:
pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 30kB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 3.9MB 49.8MB/s 
     |████████████████████████████████| 450kB 26.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=8031d151249bd67299cb4c81922e58eee7ee45e338736c7401255c396cd9a38b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tens

In [3]:
!pip install keras==2.3.1
!pip install tensorflow==2.1.0
!pip install keras_applications==1.0.8
!pip install image-classifiers==1.0.0
!pip install efficientnet==1.0.0

     |████████████████████████████████| 378kB 5.3MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [4]:
import keras
import numpy as np
import pandas as pd
import random

Using TensorFlow backend.


In [5]:
import io

In [6]:
path = keras.utils.data_utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

606208/600901 [==============================] - 0s 0us/step


In [7]:
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print(len(text))

600893


In [8]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 57


In [9]:
sentences = []
next_chars = []
for i in range(0, len(text) - 40, 3):
    sentences.append(text[i: i + 40])
    next_chars.append(text[i + 40])

x = np.zeros((len(sentences), 40, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [10]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(128, input_shape=(40, len(chars))))
model.add(keras.layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [11]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [12]:
model.fit(x, y, batch_size=18, epochs=10)
start_index = random.randint(0, len(text) - 40 - 1)
for diversity in [0.2, 0.5, 1.0, 1.2]:
    generated = ''
    sentence = text[start_index: start_index + 40]
    generated += sentence

    for i in range(400):
        x_pred = np.zeros((1, 40, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char
with open('example.txt', 'w') as f:
    f.write(generated)

Epoch 1/10
200285/200285 [==============================] - 386s 2ms/step - loss: 1.9746
Epoch 2/10
200285/200285 [==============================] - 388s 2ms/step - loss: 1.7987
Epoch 3/10
200285/200285 [==============================] - 386s 2ms/step - loss: 1.7857
Epoch 4/10
200285/200285 [==============================] - 383s 2ms/step - loss: 1.8183
Epoch 5/10
200285/200285 [==============================] - 388s 2ms/step - loss: 1.8494
Epoch 6/10
200285/200285 [==============================] - 390s 2ms/step - loss: 4.1154
Epoch 7/10
200285/200285 [==============================] - 386s 2ms/step - loss: 8.4661
Epoch 8/10
200285/200285 [==============================] - 386s 2ms/step - loss: 4.8836
Epoch 9/10
200285/200285 [==============================] - 387s 2ms/step - loss: 3.9180
Epoch 10/10
200285/200285 [==============================] - 387s 2ms/step - loss: 3.6050


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
from google.colab import files
files.download('example.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>